# Quad Music Transformer (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2024

***

# (GPU CHECK)

In [ ]:
#@title NVIDIA GPU check
!nvidia-smi

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install dependencies
!git clone --depth 1 https://github.com/asigalov61/Quad-Music-Transformer
!pip install huggingface_hub
!pip install einops
!pip install torch-summary
!apt install fluidsynth #Pip does not work for some reason. Only apt works

In [ ]:
#@title Import modules

print('=' * 70)
print('Loading core Quad Music Transformer modules...')

import os
import copy
import pickle
import secrets
import statistics
from time import time
import tqdm

print('=' * 70)
print('Loading main Quad Music Transformer modules...')
import torch

%cd /content/Quad-Music-Transformer

import TMIDIX

from midi_to_colab_audio import midi_to_colab_audio

from x_transformer_1_23_2 import *

import random

%cd /content/
print('=' * 70)
print('Loading aux Quad Music Transformer modules...')

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

from IPython.display import Audio, display

from huggingface_hub import hf_hub_download

from google.colab import files

print('=' * 70)
print('Done!')
print('Enjoy! :)')
print('=' * 70)

# (LOAD MODEL)

In [ ]:
#@title Load Quad Music Transformer Pre-Trained Model

#@markdown Model precision option

model_precision = "bfloat16" # @param ["bfloat16", "float16"]

#@markdown bfloat16 == Half precision/faster speed (if supported, otherwise the model will default to float16)

#@markdown float16 == Full precision/fast speed

plot_tokens_embeddings = False # @param {type:"boolean"}

print('=' * 70)
print('Loading Quad Music Transformer Pre-Trained Model...')
print('Please wait...')
print('=' * 70)

full_path_to_models_dir = "/content/Quad-Music-Transformer/Model/Optimized"

model_checkpoint_file_name = 'Quad_Music_Transformer_Large_Optimized_Trained_Model_47708_steps_0.7875_loss_0.7387_acc.pth'
model_path = full_path_to_models_dir+'/'+model_checkpoint_file_name
num_layers = 16

if os.path.isfile(model_path):
  print('Model already exists...')

else:
  hf_hub_download(repo_id='asigalov61/Quad-Music-Transformer',
                  filename=model_checkpoint_file_name,
                  local_dir='/content/Quad-Music-Transformer/Model/Optimized',
                  local_dir_use_symlinks=False)

print('=' * 70)
print('Instantiating model...')

device_type = 'cuda'

if model_precision == 'bfloat16' and torch.cuda.is_bf16_supported():
  dtype = 'bfloat16'
else:
  dtype = 'float16'

if model_precision == 'float16':
  dtype = 'float16'

ptdtype = {'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype)

SEQ_LEN = 8192

# instantiate the model

model = TransformerWrapper(
    num_tokens = 2598,
    max_seq_len = SEQ_LEN,
    attn_layers = Decoder(dim = 2048, depth = num_layers, heads = 16, attn_flash = True)
)

model = AutoregressiveWrapper(model, ignore_index=2597)

model.cuda()
print('=' * 70)

print('Loading model checkpoint...')

model.load_state_dict(torch.load(model_path))
print('=' * 70)

model.eval()

print('Done!')
print('=' * 70)

print('Model will use', dtype, 'precision...')
print('=' * 70)

# Model stats
print('Model summary...')
summary(model)

# Plot Token Embeddings
if plot_tokens_embeddings:
  tok_emb = model.net.token_emb.emb.weight.detach().cpu().tolist()

  cos_sim = metrics.pairwise_distances(
    tok_emb, metric='cosine'
  )
  plt.figure(figsize=(7, 7))
  plt.imshow(cos_sim, cmap="inferno", interpolation="nearest")
  im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
  plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
  plt.xlabel("Position")
  plt.ylabel("Position")
  plt.tight_layout()
  plt.plot()
  plt.savefig("/content/Quad-Music-Transformer-Tokens-Embeddings-Plot.png", bbox_inches="tight")

# (GENERATE)

# (SETUP MODEL CHANNELS MIDI PATCHES)

In [ ]:
# @title Setup and load model channels MIDI patches

model_channel_0_piano_family = "Acoustic Grand" # @param ["Acoustic Grand", "Bright Acoustic", "Electric Grand", "Honky-Tonk", "Electric Piano 1", "Electric Piano 2", "Harpsichord", "Clav"]
model_channel_1_chromatic_percussion_family = "Music Box" # @param ["Celesta", "Glockenspiel", "Music Box", "Vibraphone", "Marimba", "Xylophone", "Tubular Bells", "Dulcimer"]
model_channel_2_organ_family = "Church Organ" # @param ["Drawbar Organ", "Percussive Organ", "Rock Organ", "Church Organ", "Reed Organ", "Accordion", "Harmonica", "Tango Accordion"]
model_channel_3_guitar_family = "Acoustic Guitar(nylon)" # @param ["Acoustic Guitar(nylon)", "Acoustic Guitar(steel)", "Electric Guitar(jazz)", "Electric Guitar(clean)", "Electric Guitar(muted)", "Overdriven Guitar", "Distortion Guitar", "Guitar Harmonics"]
model_channel_4_bass_family = "Fretless Bass" # @param ["Acoustic Bass", "Electric Bass(finger)", "Electric Bass(pick)", "Fretless Bass", "Slap Bass 1", "Slap Bass 2", "Synth Bass 1", "Synth Bass 2"]
model_channel_5_strings_family = "Violin" # @param ["Violin", "Viola", "Cello", "Contrabass", "Tremolo Strings", "Pizzicato Strings", "Orchestral Harp", "Timpani"]
model_channel_6_ensemble_family = "Choir Aahs" # @param ["String Ensemble 1", "String Ensemble 2", "SynthStrings 1", "SynthStrings 2", "Choir Aahs", "Voice Oohs", "Synth Voice", "Orchestra Hit"]
model_channel_7_brass_family = "Trumpet" # @param ["Trumpet", "Trombone", "Tuba", "Muted Trumpet", "French Horn", "Brass Section", "SynthBrass 1", "SynthBrass 2"]
model_channel_8_reed_family = "Alto Sax" # @param ["Soprano Sax", "Alto Sax", "Tenor Sax", "Baritone Sax", "Oboe", "English Horn", "Bassoon", "Clarinet"]
model_channel_9_pipe_family = "Flute" # @param ["Piccolo", "Flute", "Recorder", "Pan Flute", "Blown Bottle", "Skakuhachi", "Whistle", "Ocarina"]
model_channel_10_synth_lead_family = "Lead 8 (bass+lead)" # @param ["Lead 1 (square)", "Lead 2 (sawtooth)", "Lead 3 (calliope)", "Lead 4 (chiff)", "Lead 5 (charang)", "Lead 6 (voice)", "Lead 7 (fifths)", "Lead 8 (bass+lead)"]
model_channel_11_synth_pad_family = "Pad 2 (warm)" # @param ["Pad 1 (new age)", "Pad 2 (warm)", "Pad 3 (polysynth)", "Pad 4 (choir)", "Pad 5 (bowed)", "Pad 6 (metallic)", "Pad 7 (halo)", "Pad 8 (sweep)"]
model_channel_12_synth_effects_family = "FX 3 (crystal)" # @param ["FX 1 (rain)", "FX 2 (soundtrack)", "FX 3 (crystal)", "FX 4 (atmosphere)", "FX 5 (brightness)", "FX 6 (goblins)", "FX 7 (echoes)", "FX 8 (sci-fi)"]
model_channel_13_ethnic_family = "Banjo" # @param ["Sitar", "Banjo", "Shamisen", "Koto", "Kalimba", "Bagpipe", "Fiddle", "Shanai"]
model_channel_14_percussive_family = "Melodic Tom" # @param ["Tinkle Bell", "Agogo", "Steel Drums", "Woodblock", "Taiko Drum", "Melodic Tom", "Synth Drum", "Reverse Cymbal"]
model_channel_15_sound_effects_family = "Bird Tweet" # @param ["Guitar Fret Noise", "Breath Noise", "Seashore", "Bird Tweet", "Telephone Ring", "Helicopter", "Applause", "Gunshot"]
model_channel_16_drums_family = "Drums" # @param ["Drums"]

print('=' * 70)
print('Setting up patches...')
print('=' * 70)

instruments = [v[1] for v in TMIDIX.Number2patch.items()]

patches = [instruments.index(model_channel_0_piano_family),
                       instruments.index(model_channel_1_chromatic_percussion_family),
                       instruments.index(model_channel_2_organ_family),
                       instruments.index(model_channel_3_guitar_family),
                       instruments.index(model_channel_4_bass_family),
                       instruments.index(model_channel_5_strings_family),
                       instruments.index(model_channel_6_ensemble_family),
                       instruments.index(model_channel_7_brass_family),
                       instruments.index(model_channel_8_reed_family),
                       9, # Drums patch
                       instruments.index(model_channel_9_pipe_family),
                       instruments.index(model_channel_10_synth_lead_family),
                       instruments.index(model_channel_11_synth_pad_family),
                       instruments.index(model_channel_12_synth_effects_family),
                       instruments.index(model_channel_13_ethnic_family),
                       instruments.index(model_channel_15_sound_effects_family)
                       ]

print('Done!')
print('=' * 70)

# (IMPROV)

In [ ]:
#@title Standard Improv Generator

#@markdown Improv type

improv_type = "Random Freestyle" # @param ["Random Freestyle", "Freestyle without Drums", "Freestyle with Drums", "Custom"]

#@markdown Custom Improv settings

first_note_model_channel = 0 # @param {type:"slider", min:0, max:16, step:1}
first_note_MIDI_pitch = 60 # @param {type:"slider", min:1, max:127, step:1}
add_drums = False #@param {type:"boolean"}

#@markdown Generation settings

number_of_tokens_to_generate = 918 # @param {type:"slider", min:30, max:8190, step:3}
number_of_batches_to_generate = 4 #@param {type:"slider", min:1, max:16, step:1}
temperature = 0.9 # @param {type:"slider", min:0.1, max:1, step:0.05}

#@markdown Other settings

render_MIDI_to_audio = True # @param {type:"boolean"}

print('=' * 70)
print('Quad Music Transformer Standard Improv Model Generator')
print('=' * 70)

if improv_type == 'Random Freestyle':

  outy = [2595]

if improv_type == 'Freestyle without Drums':

  outy = [2595, 2449]

if improv_type == 'Freestyle with Drums':

  outy = [2595, 2450]

if improv_type == 'Custom':

  if add_drums:
      drumsp = 2450 # Yes
  else:
      drumsp = 2449 # No

  outy = [2595, drumsp, 2450+first_note_model_channel, 2467+first_note_MIDI_pitch]

print('Selected Improv sequence:')
print(outy)
print('=' * 70)

torch.cuda.empty_cache()

inp = [outy] * number_of_batches_to_generate

inp = torch.LongTensor(inp).cuda()

with ctx:
  out = model.generate(inp,
                        number_of_tokens_to_generate,
                        temperature=temperature,
                        return_prime=True,
                        verbose=True)

out0 = out.tolist()

print('=' * 70)
print('Done!')
print('=' * 70)

torch.cuda.empty_cache()

#======================================================================

print('Rendering results...')

for i in range(number_of_batches_to_generate):

  print('=' * 70)
  print('Batch #', i)
  print('=' * 70)

  out1 = out0[i]

  print('Sample INTs', out1[:12])
  print('=' * 70)

  if len(out1) != 0:

      song = out1
      song_f = []

      time = 0
      dur = 0
      vel = 90
      pitch = 0
      channel = 0

      for ss in song:

          if 0 <= ss < 128:

              time += ss * 32

          if 128 <= ss < 256:

              dur = (ss-128) * 32

          if 256 <= ss < 2432:

              chan = (ss-256) // 128

              if chan < 9:
                  channel = chan
              elif 9 < chan < 15:
                  channel = chan+1
              elif chan == 15:
                  channel = 15
              elif chan == 16:
                  channel = 9

              pitch = (ss-256) % 128

          if 2432 <= ss < 2448:

              vel = (((ss-2432)+1) * 8)-1

              song_f.append(['note', time, dur, channel, pitch, vel, chan*8 ])

      stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Quad Music Transformer',
                                                        output_file_name = '/content/Quad-Music-Transformer-Composition_'+str(i),
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=patches
                                                        )


      print('=' * 70)
      print('Displaying resulting composition...')
      print('=' * 70)

      fname = '/content/Quad-Music-Transformer-Composition_'+str(i)

      if render_MIDI_to_audio:
        midi_audio = midi_to_colab_audio(fname + '.mid')
        display(Audio(midi_audio, rate=16000, normalize=False))

      TMIDIX.plot_ms_SONG(song_f, plot_title=fname)

# (CUSTOM MIDI)

In [ ]:
#@title Load Seed MIDI

#@markdown Press play button to to upload your own seed MIDI or to load one of the provided sample seed MIDIs from the dropdown list below

select_seed_MIDI = "Upload your own custom MIDI" # @param ["Upload your own custom MIDI", "Quad-Music-Transformer-Piano-Seed-1", "Quad-Music-Transformer-Piano-Seed-2", "Quad-Music-Transformer-Piano-Seed-3", "Quad-Music-Transformer-Piano-Seed-4", "Quad-Music-Transformer-Piano-Seed-5", "Quad-Music-Transformer-Piano-Seed-6", "Quad-Music-Transformer-MI-Seed-1", "Quad-Music-Transformer-MI-Seed-2", "Quad-Music-Transformer-MI-Seed-3", "Quad-Music-Transformer-MI-Seed-4", "Quad-Music-Transformer-MI-Seed-5", "Quad-Music-Transformer-MI-Seed-6"]
render_MIDI_to_audio = False # @param {type:"boolean"}

print('=' * 70)
print('Quad Music Transformer Seed MIDI Loader')
print('=' * 70)

f = ''

if select_seed_MIDI != "Upload your own custom MIDI":
  print('Loading seed MIDI...')
  f = '/content/Quad-Music-Transformer/Seeds/'+select_seed_MIDI+'.mid'

else:
  print('Upload your own custom MIDI...')
  print('=' * 70)
  uploaded_MIDI = files.upload()
  if list(uploaded_MIDI.keys()):
    f = list(uploaded_MIDI.keys())[0]

if f != '':

  print('=' * 70)
  print('File:', f)
  print('=' * 70)

  #=======================================================
  # START PROCESSING

  #===============================================================================
  # Raw single-track ms score

  raw_score = TMIDIX.midi2single_track_ms_score(f)

  #===============================================================================
  # Enhanced score notes

  escore_notes = TMIDIX.advanced_score_processor(raw_score, return_enhanced_score_notes=True)[0]

  if len(escore_notes) > 0:

      #=======================================================
      # PRE-PROCESSING

      # checking number of instruments in a composition
      instruments_list = list(set([y[3] for y in escore_notes]))

      #===============================================================================
      # Augmented enhanced score notes

      escore_notes = TMIDIX.augment_enhanced_score_notes(escore_notes)

      #=======================================================
      # FINAL PROCESSING

      melody_chords = []
      melody_chords2 = []

      # Break between compositions / Intro seq

      if 9 in instruments_list:
          drums_present = 2450 # Yes
      else:
          drums_present = 2449 # No

      if escore_notes[0][3] != 9:
          fpat = escore_notes[0][6]
          fptc = escore_notes[0][4]
      else:
          fpat = 128
          fptc = escore_notes[0][4]

      fchan = fpat // 8

      melody_chords.extend([2595, drums_present, 2450+fchan, 2467+fptc]) # Intro seq

      #=======================================================
      # MAIN PROCESSING CYCLE
      #=======================================================

      pe = escore_notes[0]

      chords_counter = 1

      comp_chords_len = len(list(set([y[1] for y in escore_notes])))

      for e in escore_notes:

        #=======================================================
        # Timings...

        if (e[1] - pe[1]) != 0:
            time = (e[1]-pe[1])
        else:
            time = 0

        dur = e[2]

        if time != 0 and time % 2 != 0:
            time += 1
        if dur % 2 != 0:
            dur += 1

        delta_time = int(max(0, min(255, time)) / 2)

        # Durations and channels

        dur = int(max(0, min(255, dur)) / 2)

        cha = max(0, min(15, e[3]))

        # Patches
        if cha == 9: # Drums patch will be == 128
            pat = 128

        else:
            pat = e[6]

        # Channels

        chan = pat // 8

        # Pitches

        if cha != 9:
            ptc = max(1, min(127, e[4]))
        else:
            ptc = max(1, min(127, e[4]))

        # Velocities

        # Calculating octo-velocity
        velocity = max(8, min(127, e[5]))
        vel = round(velocity / 8)-1

        #=======================================================
        # Outro seq

        # if comp_chords_len >= 250:
        #    if ((comp_chords_len - chords_counter) == 50) and (delta_time != 0):
        #        melody_chords.extend([2448, 2448, 2448, 2448]) # outro seq

        # if delta_time != 0:
        #    chords_counter += 1

        #=======================================================
        # FINAL NOTE SEQ

        # Writing final note asynchronously

        cha_ptc = (128 * chan) + ptc

        melody_chords.extend([delta_time, dur+128, cha_ptc+256, vel+2432])
        melody_chords2.append([delta_time, dur+128, cha_ptc+256, vel+2432])
        pe = e

        #=======================================================

      # melody_chords.extend([2596]) # EOS

      #=======================================================

      # TOTAL DICTIONARY SIZE 2596+1=2597
      #=======================================================

  #=======================================================

  song = melody_chords

  song_f = []

  time = 0
  dur = 0
  vel = 90
  pitch = 0
  channel = 0

  for ss in song:

      if 0 <= ss < 128:

          time += ss * 32

      if 128 <= ss < 256:

          dur = (ss-128) * 32

      if 256 <= ss < 2432:

          chan = (ss-256) // 128

          if chan < 9:
              channel = chan
          elif 9 < chan < 15:
              channel = chan+1
          elif chan == 15:
              channel = 15
          elif chan == 16:
              channel = 9

          pitch = (ss-256) % 128

      if 2432 <= ss < 2448:

          vel = (((ss-2432)+1) * 8)-1

          song_f.append(['note', time, dur, channel, pitch, vel, chan*8 ])

  stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                    output_signature = 'Quad Music Transformer',
                                                    output_file_name = '/content/Quad-Music-Transformer-Seed-Composition',
                                                    track_name='Project Los Angeles',
                                                    list_of_MIDI_patches=patches
                                                    )

  #=======================================================

  print('=' * 70)
  print('Composition stats:')
  print('Composition has', len(melody_chords2), 'notes')
  print('Composition has', len(melody_chords), 'tokens')
  print('Composition MIDI channels:', sorted(list(set([((y-256) // 128) for y in melody_chords if 256 <= y < 2432]))))
  print('=' * 70)

  print('Displaying resulting composition...')
  print('=' * 70)

  fname = '/content/Quad-Music-Transformer-Seed-Composition'

  if render_MIDI_to_audio:
    midi_audio = midi_to_colab_audio(fname + '.mid')
    display(Audio(midi_audio, rate=16000, normalize=False))

  TMIDIX.plot_ms_SONG(song_f, plot_title=fname)

else:
  print('=' * 70)

# (CONTINUATION)

In [ ]:
#@title Standard Continuation

#@markdown Generation settings

try_to_generate_outro = False #@param {type:"boolean"}
number_of_prime_tokens = 7104 # @param {type:"slider", min:4, max:8192, step:4}
number_of_tokens_to_generate = 940 # @param {type:"slider", min:40, max:8192, step:4}
number_of_batches_to_generate = 4 #@param {type:"slider", min:1, max:16, step:1}
temperature = 0.9 # @param {type:"slider", min:0.1, max:1, step:0.05}

#@markdown Other settings
include_prime_tokens_in_generated_output = True #@param {type:"boolean"}
allow_model_to_stop_generation_if_needed = False #@param {type:"boolean"}
render_MIDI_to_audio = True # @param {type:"boolean"}

print('=' * 70)
print('Quad Music Transformer Standard Continuation Model Generator')
print('=' * 70)

if allow_model_to_stop_generation_if_needed:
  min_stop_token = 2596
else:
  min_stop_token = None

outy = melody_chords[:number_of_prime_tokens]

if try_to_generate_outro:
  outy.extend([2448, 2448, 2448, 2448]) # outro seq

torch.cuda.empty_cache()

inp = [outy] * number_of_batches_to_generate

inp = torch.LongTensor(inp).cuda()

with ctx:
  out = model.generate(inp,
                        number_of_tokens_to_generate,
                        temperature=temperature,
                        return_prime=include_prime_tokens_in_generated_output,
                        eos_token=min_stop_token,
                        verbose=True)

out0 = out.tolist()

torch.cuda.empty_cache()

print('=' * 70)
print('Done!')
print('=' * 70)

#======================================================================
print('Rendering results...')

for i in range(number_of_batches_to_generate):

  print('=' * 70)
  print('Batch #', i)
  print('=' * 70)

  out1 = out0[i]

  print('Sample INTs', out1[:12])
  print('=' * 70)

  if len(out) != 0:

      song = out1
      song_f = []

      time = 0
      dur = 0
      vel = 90
      pitch = 0
      channel = 0

      for ss in song:

          if 0 <= ss < 128:

              time += ss * 32

          if 128 <= ss < 256:

              dur = (ss-128) * 32

          if 256 <= ss < 2432:

              chan = (ss-256) // 128

              if chan < 9:
                  channel = chan
              elif 9 < chan < 15:
                  channel = chan+1
              elif chan == 15:
                  channel = 15
              elif chan == 16:
                  channel = 9

              pitch = (ss-256) % 128

          if 2432 <= ss < 2448:

              vel = (((ss-2432)+1) * 8)-1

              song_f.append(['note', time, dur, channel, pitch, vel, chan*8 ])

      stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Quad Music Transformer',
                                                        output_file_name = '/content/Quad-Music-Transformer-Composition_'+str(i),
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=patches
                                                        )


      print('=' * 70)
      print('Displaying resulting composition...')
      print('=' * 70)

      fname = '/content/Quad-Music-Transformer-Composition_'+str(i)

      if render_MIDI_to_audio:
        midi_audio = midi_to_colab_audio(fname + '.mid')
        display(Audio(midi_audio, rate=16000, normalize=False))

      TMIDIX.plot_ms_SONG(song_f, plot_title=fname)

# Congrats! You did it! :)